## Data preprocess

In [129]:
import numpy as np
import pandas as pd
from pathlib import Path

def one_hot_encoding(arr):
    s = set()
    _sd = dict()
    for e in arr:
        s.add(e)

    sl = list(s)
    for i in range(len(sl)):
        encode = np.zeros(len(sl))
        encode[i] = 1
        _sd[sl[i]] = encode

    encoded_data = []
    for k in range(len(arr)):
        encoded_data.append(_sd[arr[k]])

    return np.array(encoded_data)


def data_preprocess(filepath):
    df = pd.read_excel(filepath)
    data = df.to_numpy()

    mask = np.ones(data.shape[1], dtype=bool)
    mask[2] = False

    expanded_data = data[:, mask]

    for i in range(len(expanded_data)):
        for j in range(len(expanded_data[i])):
            expanded_data[i][j] = float(expanded_data[i][j])

    return expanded_data


filename = "reuqestNumber_processTime_Prediction"

resutls_dir = Path.cwd().parent / "results"

# Data preprocessing
file_path = f"{resutls_dir}\\{filename}.xlsx"
dataset = data_preprocess(file_path)

print(dataset.shape, '\n', dataset[0])

(50, 3) 
 [55.04470920562744 280807.0 3.543741464614868]


## XGBOOST - 1

In [130]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# X request number
# y process time


X, y = dataset[:, 1:2], dataset[:, -1:]

print(X.shape, X[0])
print(y.shape, y[0])


train_size = int(len(X) * 0.7)

# 拆分数据集为训练集和测试集
# split dataset to train dataset and test dataset

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# time series
# X_train = X[:train_size]
# X_test = X[train_size:]
# y_train = y[:train_size]
# y_test = y[train_size:]

(50, 1) [280807.0]
(50, 1) [3.543741464614868]


In [131]:
from xgboost import XGBRegressor

# Create XGBoost regression model
model = XGBRegressor(
    objective="reg:squarederror",   # Loss function to minimize: squared error for regression  损失函数为平方损失函数, 
    n_estimators=100,  # Number of boosting rounds (number of trees)  迭代次数，即基础学习器的数量
    learning_rate=0.1,  # Boosting learning rate (controls the step size)  学习率，控制每次更新的步长
    max_depth=5,  # Maximum depth of a tree  树的最大深度
    min_child_weight=1,  # Minimum sum of instance weight (hessian) needed in a child  叶子节点最小权重
    subsample=0.8,  # Subsample ratio of the training instance (randomly sampled)  每棵树随机采样的比例
    colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree  每棵树随机选择的特征比例
    alpha=0.01,  # L1 regularization term on weights  L1 正则化项参数
    reg_lambda=0.01,  # L2 regularization term on weights  L2 正则化项参数
)

# model fit
model.fit(X_train, y_train)

XGBRegressor(alpha=0.01, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [132]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mea = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mea}")


print(f"    real    |    pred   \n")
for i in range(len(y_test)):
    print(f"    {y_test[i]}     |     {y_pred[i]}      ")

Mean Squared Error: 0.4054700155691234
Mean Absolute Error: 0.4191955778747797
    real    |    pred   

    [13.60307049751282]     |     13.102801322937012      
    [20.29998636245728]     |     18.56572151184082      
    [12.38283538818359]     |     11.94563102722168      
    [17.12697076797485]     |     16.592073440551758      
    [3.612605333328247]     |     3.5185978412628174      
    [15.84868288040161]     |     15.979058265686035      
    [0.2680389881134033]     |     0.12099160254001617      
    [0.8576817512512207]     |     0.8429433107376099      
    [0.1278862953186035]     |     0.0431676022708416      
    [17.10650610923767]     |     16.592073440551758      


## XGBRegressor

In [133]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error


# 创建 XGBoost 回归模型
model = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    learning_rate=0.1,
    tree_method="hist",
    device="cuda",
)

# Define the parameter grid for GridSearchCV
param_grid = {
    "max_depth": [3],  # Range of maximum depth of trees to explore  扩展最大深度的范围
    "learning_rate": [0.1],  # Range of learning rates to explore  扩展学习率的范围
    "n_estimators": [100],  # Range of number of trees (boosting rounds) to explore  扩展树的数量范围
    "gamma": [0],   # Range of gamma parameter to explore (controls tree split) 扩展 gamma 参数范围
    "subsample": [1],  # Range of subsample ratios to explore  扩展子样本比例范围
    "colsample_bytree": [0.8, 0.7],  # Range of column subsample ratios for each tree to explore  扩展列采样比例范围
    "reg_alpha": [0, 0.5],  # Range of L1 regularization parameters to explore  扩展 L1 正则化参数范围
    "reg_lambda": [0, 0.5],  # Range of L2 regularization parameters to explore  扩展 L2 正则化参数范围
}


grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error"
)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)

# 使用最佳参数重新训练模型
# Re-train the model using the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train, verbose=True)


Best parameters found:  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 0, 'subsample': 1}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [148]:
from openpyxl import Workbook
from pathlib import Path

def to_excel(data, filename, sheet_style):
    workbook = Workbook()
    sheet = workbook.active

    # | user-response-time | request-number | response-ip   | process-time  |
    # | 0.3                | 10000          | 192.168.0.150 | 14.523432     |
    # | 0.5                | 20000          | 192.168.0.151 | 9.5232642     |

    sheet.append(sheet_style)

    for row in data:
        sheet.append(row)

    results_dir = str(Path.cwd().parent / "results")

    workbook.save(filename=f"{results_dir}\\{filename}.xlsx")

In [152]:
preds = best_model.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE: {rmse}")

mae = mean_absolute_error(y_test, preds)
print(f"MAE: {mae}")


avg = np.mean(y_test)

print(f"AVG: {avg}")

# write into excal

sheet_style = ["test", "prediction", "difference", "accuracy"]

data_list = list()

a = 0
for i in range(len(y_test)):
    if (abs(y_test[i] - y_pred[i])) <= 1:
        a += 1

    print(data_list)
    data_list.append(
        [
            y_test[i][0],
            y_pred[i],
            abs(y_test[i] - y_pred[i])[0],
            1 - (abs(y_test[i] - y_pred[i]) / y_test[i])[0],
        ],
    )


print(f"Accuracy in 500ms: {round(100 * a / len(y_test), 3)}%")

print(data_list[-1])

filename = "reuqestNumber_processTime_Prediction"
to_excel(data_list, filename, sheet_style)


print("Results saved finished.")

RMSE: 0.6400760762797237
MAE: 0.4202775001525879
AVG: 10.12342643737793
[]
[[13.60307049751282, 13.102801, 0.5002691745758074, 0.9632238048999837]]
[[13.60307049751282, 13.102801, 0.5002691745758074, 0.9632238048999837], [20.29998636245728, 18.565722, 1.7342648506164586, 0.9145681765666699]]
[[13.60307049751282, 13.102801, 0.5002691745758074, 0.9632238048999837], [20.29998636245728, 18.565722, 1.7342648506164586, 0.9145681765666699], [12.38283538818359, 11.945631, 0.4372043609619105, 0.964692709928203]]
[[13.60307049751282, 13.102801, 0.5002691745758074, 0.9632238048999837], [20.29998636245728, 18.565722, 1.7342648506164586, 0.9145681765666699], [12.38283538818359, 11.945631, 0.4372043609619105, 0.964692709928203], [17.12697076797485, 16.592073, 0.5348973274230922, 0.9687687137048614]]
[[13.60307049751282, 13.102801, 0.5002691745758074, 0.9632238048999837], [20.29998636245728, 18.565722, 1.7342648506164586, 0.9145681765666699], [12.38283538818359, 11.945631, 0.4372043609619105, 0.96469

d:\model_fit\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
